In [ ]:
!pip install qiskit

In [ ]:
import os
import csv
from os.path import join, isfile

from qiskit import QuantumCircuit, transpile
from qiskit.qasm2 import dumps

In [ ]:
def qiskit_optimize_ibm(fn, output_dir: str):
    circ = QuantumCircuit.from_qasm_file(fn)

    optimized_circ = transpile(
        circ, basis_gates=["rz", "cx", "h", "x"], optimization_level=3
    )

    count_dict = optimized_circ.count_ops()
    output_path = join(output_dir, os.path.basename(fn))

    with open(output_path, "w") as f:
        f.write(dumps(optimized_circ))

    return (
        sum(count_dict.values()),
        count_dict.get("cx", 0),
        optimized_circ.depth(),
    )


circuit_order = [
    "tof_3",
    "barenco_tof_3",
    "mod5_4",
    "tof_4",
    "tof_5",
    "barenco_tof_4",
    "mod_mult_55",
    "vbe_adder_3",
    "barenco_tof_5",
    "csla_mux_3",
    "rc_adder_6",
    "gf2^4_mult",
    "hwb6",
    "mod_red_21",
    "tof_10",
    "gf2^5_mult",
    "csum_mux_9",
    "barenco_tof_10",
    "ham15-low",
    "qcla_com_7",
    "gf2^6_mult",
    "qcla_adder_10",
    "gf2^7_mult",
    "gf2^8_mult",
    "qcla_mod_7",
    "adder_8",
    "vqe_nativegates_ibm_tket_8",
    "qgan_nativegates_ibm_tket_8",
    "qaoa_nativegates_ibm_tket_8",
    "ae_nativegates_ibm_tket_8",
    "qpeexact_nativegates_ibm_tket_8",
    "qpeinexact_nativegates_ibm_tket_8",
    "qft_nativegates_ibm_tket_8",
    "qftentangled_nativegates_ibm_tket_8",
    "portfoliovqe_nativegates_ibm_tket_8",
    "portfolioqaoa_nativegates_ibm_tket_8",
]

path = "../circuits/ibm/"

fns = [
    fn for fn in os.listdir(path) if isfile(join(path, fn)) and fn[-4:] == "qasm"
]

extra_path = "../circuits/ibm_extra/"
extra_fns = [
    fn
    for fn in os.listdir(extra_path)
    if isfile(join(extra_path, fn)) and fn[-4:] == "qasm"
]

all_fns = fns + extra_fns
all_paths = [path] * len(fns) + [extra_path] * len(extra_fns)

output_dir = "./output/qiskit_ibm"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

result_dict = {}

for i, (fn, path) in enumerate(zip(all_fns, all_paths)):
    if fn[:-5] not in circuit_order:
        continue

    gate_cnt, cnot_cnt, depth = qiskit_optimize_ibm(
        path + fn, output_dir
    )

    result_dict[fn] = [gate_cnt, cnot_cnt, depth]

csv_file = "./metricks/qiskit_ibm.csv"

with open(csv_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "total_gates", "cx_gates", "depth"])
    for fn in circuit_order:
        if fn + ".qasm" in result_dict:
            writer.writerow([fn + ".qasm"] + result_dict[fn + ".qasm"])
